## Code development to create a national table

https://mapbox.github.io/rasterio/api/rasterio.mask.html?highlight=mask#module-rasterio.mask

http://thematicmapping.org/downloads/world_borders.php

http://www.gadm.org

https://github.com/mapbox/rasterio-cookbook/blob/master/recipies/mask_shp.py


http://gis.stackexchange.com/questions/151339/rasterize-a-shapefile-with-geopandas-or-fiona-python


## Necessary steps (note, this will also be needed in HELIX project...)

* Extract single feature (country shape)
* Reproject feature to match raster
* Convert vector to raster
* Mask original raster by converted raster
* Calculate statistics over mask
* Add to dictionary.
* Repeat for every feature

In [ ]:
import fiona
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Read shape files using Geopandas

In [ ]:
nations = gpd.read_file('/Users/Ben/Downloads/TM_WORLD_BORDERS-0/TM_WORLD_BORDERS-0.3.shp')

In [ ]:
nations.head()

In [ ]:
nations[nations.ISO3 == "GBR"]

In [ ]:
nations[nations.ISO3 == "GBR"].plot()

In [ ]:
uk = nations[nations.ISO3 == "GBR"].geometry

### Rasterise

* Use a raster file to extract metadata and use as a template

In [ ]:
from rasterio import features

In [ ]:
rst_fn = '../eurolst_process/processed.tif'
rst = rasterio.open(rst_fn)
meta = rst.meta.copy()
meta.update(compress='lzw')

In [ ]:
meta

In [ ]:
with rasterio.open(out_fn, 'w', **meta) as out:
    out_arr = out.read(1)

In [ ]:
burned = features.rasterize(shapes=uk, fill=0, out=out_arr, transform=meta['transform'])

In [ ]:
uk

In [ ]:
from shapely.geometry import Point

In [ ]:
for boundary in uk.boundary:
    bound = boundary

In [ ]:
for xy in bound:
    print(xy)

In [ ]:
for x, y in xy.coords:
    print(x, y)

In [ ]:
import pprint

In [ ]:
with fiona.open("/Users/Ben/Downloads/TM_WORLD_BORDERS-0/TM_WORLD_BORDERS-0.3.dbf", "r") as shapefile:
    geoms = [feature["geometry"] for feature in shapefile]
    props = [feature['properties'] for feature in shapefile ]


In [ ]:
geoms[0]

In [ ]:
props[0]

In [ ]:
with fiona.open('/Users/Ben/Downloads/TM_WORLD_BORDERS-0/TM_WORLD_BORDERS-0.3.dbf') as src:
    polydata = src
    pprint.pprint(polydata[1])

In [ ]:
# out_meta.update({"driver": "GTiff",
#                  "height": out_image.shape[1],
#                  "width": out_image.shape[2],
#                  "transform": out_transform})

# with rasterio.open("/tmp/masked.tif", "w", **out_meta) as dest:
#     dest.write(out_image)

In [ ]:
with rasterio.open("../eurolst_process/processed.tif", blockxsize=256, blockysize=256) as inData:
    window = (6000, 12000), (6000, 12000)
    window2 = (6000, 8000), (6000, 8000)
    profile = inData.profile
    cmap = plt.get_cmap('jet')
    dslice = inData.read(1, window=window, masked=False)
    show(inData.read(1, window=window, masked=True), cmap=cmap, interpolation='none')

In [ ]:
# geoms = nations[nations.ISO3 == "GBR"].geometry

# with rasterio.open("../eurolst_process/processed.tif") as src:
#     out_image, out_transform = mask(src, geoms, crop=True)
#     out_meta = src.meta.copy()

# # out_meta.update({"driver": "GTiff",
# #                  "height": out_image.shape[1],
# #                  "width": out_image.shape[2],
# #                  "transform": out_transform})

# # with rasterio.open("../eurolst_process/masked.tif", "w", **out_meta) as dest:
# #     dest.write(out_image)